In [53]:
import pandas as pd
import numpy as np

In [54]:
up1 = pd.read_csv('./trajectory/up1.csv')
up2 = pd.read_csv('./trajectory/up2.csv')
right1 = pd.read_csv('./trajectory/right1.csv')
right2 = pd.read_csv('./trajectory/right2.csv')
down1 = pd.read_csv('./trajectory/down1.csv')
down2 = pd.read_csv('./trajectory/down2.csv')

up1 = np.array(up1)
up2 = np.array(up2)
right1 = np.array(right1)
right2 = np.array(right2)
down1 = np.array(down1)
down2 = np.array(down2)

In [55]:
up1 = up1.reshape(len(up1), 3)
up2 = up2.reshape(len(up2), 3)
right1 = right1.reshape(len(right1), 3)
right2 = right2.reshape(len(right2), 3)
down1 = down1.reshape(len(down1), 3)
down2 = down2.reshape(len(down2), 3)

In [56]:
seq = np.vstack([up1, up2])
seq = np.append(seq, right1, axis=0)
seq = np.append(seq, right2, axis=0)
seq = np.append(seq, down1, axis=0)
seq = np.append(seq, down1, axis=0)
seq.shape

(210, 3)

In [57]:
x_max = max(seq[:,0])
x_min = min(seq[:,0])
y_max = max(seq[:,1])
y_min = min(seq[:,1])
z_max = max(seq[:,2])
z_min = min(seq[:,2])
print(x_min, x_max, y_min, y_max, z_min, z_max)

66.4967 95.4134 -15.1893 278.767 1.4344700000000001 58.4673


In [58]:
seq = seq.reshape(6, 35, 3)
seq

array([[[ 80.081    , 278.725    ,  51.5481   ],
        [ 79.809    , 268.847    ,  53.0291   ],
        [ 79.538    , 257.158    ,  54.7825   ],
        [ 79.2844   , 245.89     ,  55.9122   ],
        [ 78.8713   , 239.182    ,  56.0064   ],
        [ 78.4739   , 232.452    ,  55.5936   ],
        [ 78.259    , 218.466    ,  56.5888   ],
        [ 77.8914   , 208.481    ,  55.9948   ],
        [ 77.5442   , 200.509    ,  54.8337   ],
        [ 77.3829   , 189.691    ,  54.3085   ],
        [ 77.0679   , 180.679    ,  52.9485   ],
        [ 76.7701   , 171.867    ,  51.232    ],
        [ 76.648    , 162.067    ,  49.7529   ],
        [ 76.5374   , 152.249    ,  47.2838   ],
        [ 76.2805   , 144.166    ,  44.7854   ],
        [ 75.8809   , 138.212    ,  40.8821   ],
        [ 75.812    , 128.027    ,  38.242    ],
        [ 75.5966   , 120.273    ,  34.8816   ],
        [ 75.3965   , 112.388    ,  30.8262   ],
        [ 75.2178   , 103.768    ,  27.0571   ],
        [ 75.1927   

In [59]:
tmp = np.zeros((6,35,3))

In [60]:
for i in range(seq.shape[0]):
    for j in range(seq.shape[1]):
        tmp[i,j,0] = (seq[i,j,0]-x_min)/(x_max-x_min)
        tmp[i,j,1] = (seq[i,j,1]-y_min)/(y_max-y_min)
        tmp[i,j,2] = (seq[i,j,2]-z_min)/(z_max-z_min)
        
tmp

array([[[0.46977352, 0.99985712, 0.8786804 ],
        [0.46036719, 0.96625349, 0.9046479 ],
        [0.45099545, 0.92648907, 0.9353916 ],
        [0.44222543, 0.88815685, 0.95519949],
        [0.42793956, 0.86533713, 0.95685117],
        [0.41419664, 0.84244257, 0.94961323],
        [0.40676495, 0.79486407, 0.96706283],
        [0.39405257, 0.76089643, 0.95664778],
        [0.38204567, 0.73377676, 0.93628933],
        [0.37646758, 0.69697537, 0.9270806 ],
        [0.36557422, 0.66631775, 0.90323468],
        [0.35527567, 0.6363405 , 0.87313798],
        [0.3510532 , 0.60300221, 0.84720379],
        [0.34722842, 0.56960269, 0.80391119],
        [0.33834428, 0.54210541, 0.76010484],
        [0.32452527, 0.5218507 , 0.69166531],
        [0.32214257, 0.48720269, 0.64537443],
        [0.31469359, 0.46082462, 0.58645398],
        [0.30777371, 0.4340009 , 0.51534756],
        [0.30159389, 0.40467682, 0.44926107],
        [0.30072588, 0.37262954, 0.3805936 ],
        [0.29531032, 0.34523091, 0

In [61]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

In [62]:
hidden_layer = 256
n_step = 35
features = 3

In [63]:
model = Sequential()
model.add(LSTM(hidden_layer, activation='relu', input_shape=(n_step, features)))
model.add(RepeatVector(n_step))
model.add(LSTM(hidden_layer, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(3)))

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 256)               266240    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 35, 256)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 35, 256)           525312    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 35, 3)             771       
Total params: 792,323
Trainable params: 792,323
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [66]:
model.fit(tmp, tmp, batch_size=3, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 165ms/step - loss: 0.2720 - acc: 0.6714
Epoch 2/600
6/6 [==============================] - 0s 16ms/step - loss: 0.2386 - acc: 0.6714
Epoch 3/600
6/6 [==============================] - 0s 16ms/step - loss: 0.1951 - acc: 0.6714
Epoch 4/600
6/6 [==============================] - 0s 17ms/step - loss: 0.1399 - acc: 0.6714
Epoch 5/600
6/6 [==============================] - 0s 16ms/step - loss: 0.1425 - acc: 0.6714
Epoch 6/600
6/6 [==============================] - 0s 17ms/step - loss: 0.1203 - acc: 0.6714
Epoch 7/600
6/6 [==============================] - 0s 16ms/step - loss: 0.1226 - acc: 0.6714
Epoch 8/600
6/6 [==============================] - 0s 16ms/step - loss: 0.1177 - acc: 0.6714
Epoch 9/600
6/6 [==============================] - 0s 16ms/step - loss: 0.1030 - acc: 0.6714
Epoch 10/600
6/6 [==============================] - 0s 17ms/step - loss: 0.0931 - acc: 0.6714
Epoch 11/600
6/6 [==============================] - 0s 17ms/step - l

6/6 [==============================] - 0s 17ms/step - loss: 0.0079 - acc: 0.8381
Epoch 89/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0079 - acc: 0.8238
Epoch 90/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0068 - acc: 0.8667
Epoch 91/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0062 - acc: 0.8571
Epoch 92/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0059 - acc: 0.8571
Epoch 93/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0052 - acc: 0.8476
Epoch 94/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0055 - acc: 0.9000
Epoch 95/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0049 - acc: 0.8905
Epoch 96/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0043 - acc: 0.8667
Epoch 97/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0052 - acc: 0.8714
Epoch 98/600
6/6 [==============================] - 0s 15ms/step - loss: 

6/6 [==============================] - 0s 17ms/step - loss: 0.0024 - acc: 0.9286
Epoch 175/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0016 - acc: 0.9238
Epoch 176/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0026 - acc: 0.9095
Epoch 177/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0026 - acc: 0.8952
Epoch 178/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0016 - acc: 0.9286
Epoch 179/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0017 - acc: 0.9190
Epoch 180/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0015 - acc: 0.9095
Epoch 181/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0017 - acc: 0.9190
Epoch 182/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0014 - acc: 0.9381
Epoch 183/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0014 - acc: 0.9286
Epoch 184/600
6/6 [==============================] - 0s 16ms/ste

6/6 [==============================] - 0s 16ms/step - loss: 4.7744e-04 - acc: 0.9619
Epoch 259/600
6/6 [==============================] - 0s 17ms/step - loss: 4.7580e-04 - acc: 0.9524
Epoch 260/600
6/6 [==============================] - 0s 16ms/step - loss: 4.5454e-04 - acc: 0.9571
Epoch 261/600
6/6 [==============================] - 0s 16ms/step - loss: 4.3350e-04 - acc: 0.9571
Epoch 262/600
6/6 [==============================] - 0s 15ms/step - loss: 4.5689e-04 - acc: 0.9619
Epoch 263/600
6/6 [==============================] - 0s 16ms/step - loss: 4.6332e-04 - acc: 0.9429
Epoch 264/600
6/6 [==============================] - 0s 16ms/step - loss: 5.2267e-04 - acc: 0.9571
Epoch 265/600
6/6 [==============================] - 0s 16ms/step - loss: 4.7386e-04 - acc: 0.9429
Epoch 266/600
6/6 [==============================] - 0s 16ms/step - loss: 4.8712e-04 - acc: 0.9571
Epoch 267/600
6/6 [==============================] - 0s 16ms/step - loss: 4.5404e-04 - acc: 0.9524
Epoch 268/600
6/6 [=====

6/6 [==============================] - 0s 15ms/step - loss: 0.0012 - acc: 0.9286
Epoch 342/600
6/6 [==============================] - 0s 16ms/step - loss: 9.5491e-04 - acc: 0.9286
Epoch 343/600
6/6 [==============================] - 0s 16ms/step - loss: 9.2404e-04 - acc: 0.9619
Epoch 344/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0010 - acc: 0.9286
Epoch 345/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0012 - acc: 0.9238
Epoch 346/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0013 - acc: 0.9095
Epoch 347/600
6/6 [==============================] - 0s 16ms/step - loss: 7.8430e-04 - acc: 0.9524
Epoch 348/600
6/6 [==============================] - 0s 16ms/step - loss: 8.9935e-04 - acc: 0.9429
Epoch 349/600
6/6 [==============================] - 0s 16ms/step - loss: 7.0228e-04 - acc: 0.9429
Epoch 350/600
6/6 [==============================] - 0s 16ms/step - loss: 8.9843e-04 - acc: 0.9381
Epoch 351/600
6/6 [=====================

6/6 [==============================] - 0s 16ms/step - loss: 6.5668e-04 - acc: 0.9571
Epoch 425/600
6/6 [==============================] - 0s 16ms/step - loss: 5.8966e-04 - acc: 0.9143
Epoch 426/600
6/6 [==============================] - 0s 15ms/step - loss: 4.1737e-04 - acc: 0.9714
Epoch 427/600
6/6 [==============================] - 0s 16ms/step - loss: 3.1081e-04 - acc: 0.9333
Epoch 428/600
6/6 [==============================] - 0s 16ms/step - loss: 3.0106e-04 - acc: 0.9714
Epoch 429/600
6/6 [==============================] - 0s 16ms/step - loss: 3.1005e-04 - acc: 0.9476
Epoch 430/600
6/6 [==============================] - 0s 16ms/step - loss: 2.9537e-04 - acc: 0.9714
Epoch 431/600
6/6 [==============================] - 0s 16ms/step - loss: 2.9231e-04 - acc: 0.9286
Epoch 432/600
6/6 [==============================] - 0s 15ms/step - loss: 2.8048e-04 - acc: 0.9619
Epoch 433/600
6/6 [==============================] - 0s 15ms/step - loss: 2.7585e-04 - acc: 0.9524
Epoch 434/600
6/6 [=====

6/6 [==============================] - 0s 16ms/step - loss: 4.5094e-04 - acc: 0.9762
Epoch 507/600
6/6 [==============================] - 0s 16ms/step - loss: 4.4441e-04 - acc: 0.9524
Epoch 508/600
6/6 [==============================] - 0s 16ms/step - loss: 3.7117e-04 - acc: 0.9714
Epoch 509/600
6/6 [==============================] - 0s 16ms/step - loss: 3.0022e-04 - acc: 0.9238
Epoch 510/600
6/6 [==============================] - 0s 17ms/step - loss: 2.5762e-04 - acc: 0.9667
Epoch 511/600
6/6 [==============================] - 0s 16ms/step - loss: 3.5393e-04 - acc: 0.9524
Epoch 512/600
6/6 [==============================] - 0s 16ms/step - loss: 3.9478e-04 - acc: 0.9762
Epoch 513/600
6/6 [==============================] - 0s 16ms/step - loss: 3.5454e-04 - acc: 0.9238
Epoch 514/600
6/6 [==============================] - 0s 16ms/step - loss: 4.7361e-04 - acc: 0.9667
Epoch 515/600
6/6 [==============================] - 0s 17ms/step - loss: 5.3671e-04 - acc: 0.9333
Epoch 516/600
6/6 [=====

6/6 [==============================] - 0s 16ms/step - loss: 1.4009e-04 - acc: 0.9571
Epoch 589/600
6/6 [==============================] - 0s 16ms/step - loss: 1.3945e-04 - acc: 0.9571
Epoch 590/600
6/6 [==============================] - 0s 15ms/step - loss: 1.3500e-04 - acc: 0.9619
Epoch 591/600
6/6 [==============================] - 0s 16ms/step - loss: 1.5709e-04 - acc: 0.9762
Epoch 592/600
6/6 [==============================] - 0s 16ms/step - loss: 1.3921e-04 - acc: 0.9571
Epoch 593/600
6/6 [==============================] - 0s 16ms/step - loss: 1.4869e-04 - acc: 0.9619
Epoch 594/600
6/6 [==============================] - 0s 16ms/step - loss: 1.5061e-04 - acc: 0.9667
Epoch 595/600
6/6 [==============================] - 0s 15ms/step - loss: 1.4283e-04 - acc: 0.9714
Epoch 596/600
6/6 [==============================] - 0s 16ms/step - loss: 1.4211e-04 - acc: 0.9571
Epoch 597/600
6/6 [==============================] - 0s 17ms/step - loss: 1.4018e-04 - acc: 0.9714
Epoch 598/600
6/6 [=====

In [67]:
yhat = model.predict(tmp)

In [68]:
pred = yhat
for i in range(yhat.shape[0]):
    for j in range(yhat.shape[1]):
        pred[i,j,0] = pred[i,j,0]*(x_max-x_min)+x_min
        pred[i,j,1] = pred[i,j,1]*(y_max-y_min)+y_min
        pred[i,j,2] = pred[i,j,2]*(z_max-z_min)+z_min
pred

array([[[ 80.19032   , 287.6189    ,  54.85453   ],
        [ 79.81808   , 273.03738   ,  54.736324  ],
        [ 79.382965  , 264.3934    ,  55.553265  ],
        [ 79.07502   , 253.42412   ,  55.89641   ],
        [ 78.7258    , 241.62941   ,  56.1614    ],
        [ 78.40736   , 230.45805   ,  56.30958   ],
        [ 78.089874  , 219.78224   ,  56.219543  ],
        [ 77.77251   , 209.48166   ,  55.793232  ],
        [ 77.47267   , 199.67531   ,  54.985783  ],
        [ 77.18038   , 190.05034   ,  53.77752   ],
        [ 76.9529    , 181.29726   ,  52.362682  ],
        [ 76.75408   , 172.56398   ,  50.647102  ],
        [ 76.538635  , 163.57256   ,  48.625935  ],
        [ 76.30454   , 154.31345   ,  46.3234    ],
        [ 76.06249   , 145.02983   ,  43.783585  ],
        [ 75.81109   , 135.80342   ,  41.04731   ],
        [ 75.56617   , 126.614334  ,  38.14921   ],
        [ 75.3402    , 117.63348   ,  35.13969   ],
        [ 75.18394   , 109.52795   ,  31.869488  ],
        [ 75

In [69]:
error = np.zeros((seq.shape[0], seq.shape[1], seq.shape[2]))
for i in range(error.shape[0]):
    for j in range(error.shape[1]):
        for k in range(error.shape[2]):
            error[i,j,k] = seq[i,j,k]-pred[i,j,k]

np.set_printoptions(suppress=True)
error

array([[[ -0.10932288,  -8.89389648,  -3.30643033],
        [ -0.00907709,  -4.19038403,  -1.70722431],
        [  0.15503491,  -7.2354021 ,  -0.77076462],
        [  0.20938016,  -7.53411804,   0.0157881 ],
        [  0.14550044,  -2.44740979,  -0.15499984],
        [  0.06654374,   1.99394641,  -0.71598176],
        [  0.16912573,  -1.31624182,   0.36925654],
        [  0.11889329,  -1.00065894,   0.20156804],
        [  0.07152849,   0.83369177,  -0.15208262],
        [  0.20251823,  -0.35933875,   0.53098077],
        [  0.11499625,  -0.61825647,   0.58581766],
        [  0.01601827,  -0.6969801 ,   0.58489764],
        [  0.10936475,  -1.50555554,   1.1269654 ],
        [  0.23285746,  -2.06444604,   0.96040141],
        [  0.21800763,  -0.86383093,   1.00181541],
        [  0.06981144,   2.40857898,  -0.16520988],
        [  0.24583026,   1.41266589,   0.09278812],
        [  0.25639767,   2.63952374,  -0.2580904 ],
        [  0.21256293,   2.86005353,  -1.04328776],
        [  0